<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_4/02_fine_tuning_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ファインチューニングによる感情分析
ファインチューニングを活用し、文章の好悪感情を判別できるようにモデルを訓練します。

## ライブラリのインストール
ライブラリTransformers、およびnlpをインストールします。  

In [ ]:
!pip install transformers
!pip install nlp
!pip install dill==0.3.5
!pip install accelerate

## モデルとTokenizerの読み込み
事前学習済みのモデルと、これと紐づいたTokenizerを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast

sc_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
sc_model.cuda()
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

## データセットの読み込み
ライブラリnlpを使用して、IMDbデータセットを読み込みます。  
IMDbデータセットは、25000の映画レビューコメントに、ポジティブかネガティブの好悪感情を表すラベルが付随した、感情分析用のデータセットです。  
https://www.imdb.com/interfaces/  
読み込んだIMDbのデータはトークナイザーで処理し、形式を整えます。

In [ ]:
from nlp import load_dataset

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

train_data, test_data = load_dataset("imdb", split=["train", "test"])
print(train_data["label"][0], train_data["text"][0])  # 好意的なコメント
print(train_data["label"][20000], train_data["text"][20000])  # 否定的なコメント

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

test_data = test_data.map(tokenize, batched=True, batch_size=len(train_data))
test_data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

## 評価用の関数
`sklearn.metrics`を使用し、モデルを評価するための関数を定義します。  


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

## Trainerの設定
Trainerクラス、およびTrainingArgumentsクラスを使用して、訓練を行うTrainerの設定を行います。
https://huggingface.co/transformers/main_classes/trainer.html   
https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments  

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    per_gpu_train_batch_size = 8,
    warmup_steps = 500,  # 学習係数が0からこのステップ数で上昇
    weight_decay = 0.01,  # 重みの減衰率
    # evaluate_during_training = True,  # ここの記述はバージョンによっては必要ありません
    logging_dir = "./logs",
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data
)

## モデルの訓練
設定に基づきモデルを訓練します。


In [ ]:
trainer.train()

Step,Training Loss
500,0.455500
1000,0.359300


## モデルの評価
Trainerの`evaluate()`メソッドによりモデルを評価します。

In [ ]:
trainer.evaluate()

## TensorBoardによる結果の表示
TensorBoardを使って、logsフォルダに格納された学習過程を表示します。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs